# Abstract 

La posibilidad que brindan los sistemas automatizados de decision y clasificacion son sobresalientes al producir mejoras de producto y reducir costos o aumentar ganacias, de manera sistematica y eficaz. Muchas empresas relacionadas al juego y apuestas utilizan esto para predecir numeros ante la gran cantidad de datos que se nos brinda dados en el dataset. Asi mismo , el cliente propone una solucion practica ante las circunstacias economicas que atraviesa.

En base a esta informacion , se puede hacer una investigacion y resolver las practicas beneficiosas de los jugadores y del equipo en cada uno de los juegos, es decir , es posible hacer preguntas y responderlas para obtener nuevos conocimientos sobre los equipos, los jugadores y la competencia en sí. Por ejemplo, se puede analizar la tendencia de los equipos en casa y fuera de casa, las estadísticas de los jugadores en diferentes momentos del partido, o el porcentaje de victorias y derrotas de los equipos.

Este proyecto se basara en utilizar los datos disponibles para encontrar las diferencias y probrabilidades en cada partido dado durante los años jugados por la NBA. El objetivo se centrara en poder adquirir una variable que determine un numero en el cual se basara la cuota cobrada por el sistema de apuestas. Ademas , este analisis proovera al cliente con el gran beneficio del conocimiento y la ventaja por sobre la competencia , o hacia los mismos usuarios de la plataforma.

Asi, lo visto durante un analisis exploratorio de los datos y las caracteristicas del conjunto , se plantearan las primeras hipotesis para luego establecer un modelo predictivo de los datos de interes. Luego de un resultado final , se podra verificar el uso de las herramientas y las capacidades un sistema de ML.

## Analisis exploratorio complementario (Adquisicion de datos por APIs)

Para verificar con mayor veracidad los datos , podria utilizarse alguna API que nos permita obtener datos actualizados acerca de los juegos que no se encuentran en el dataset. Estos datos nuevos podrian utilizarse en la parte de validacion del modelo o para sacar nuevas hipotesis en base a las tendencias de los nuevos juegos. 

Hay que comentar que estos tipos de datos suelen ser de pago y no siempre se encuentran de manera libre o por "suscripcion gratuita". Lo ideal seria utilizar APIs que sean oficiales , limpias de datos erroneos o ordenados de manera correcta y sistematica.

En esta oportunidad utilizaremos la API publica brindada por *"Ball don't lie"* , que a pesar de ser una API NO OFICIAL, nos permitira obtener algunos datos relevantes de las temporadas que no tenemos en el dataset de nuestro proyecto. Su website es "https://www.balldontlie.io/home.html#introduction" en la que se encuentra la manera de realizar "querys" a su url.

Recuerde que los datos del dataset provisto durante las etapas anteriores , alcanzan las temporadas 2003-2021 (Ultima fecha registrada = 2022-03-12)

Como se nombro anteriormente , esta informacion se utilizara para complementar los datos de las temporadas anteriores , por lo tanto , se procedera a realizar un "GET" para la informacion de la temporadara ("SEASON") 2022.

In [52]:
import requests

url = "https://www.balldontlie.io/api/v1/games?seasons[]=2022&per_page=100"

response = requests.get(url)
data = response.json()
data['meta']

{'total_pages': 14,
 'current_page': 1,
 'next_page': 2,
 'per_page': 100,
 'total_count': 1307}

Al ser demasiada informacion para procesar , utilizan un sistema de paginas que proveen la informacion por "bloques". En el apartado ['meta'] , nos provee de la informacion que se obtiene del uso de la API , que nos informa que existen **16 paginas** y **1307 claves** para esta temporada.

Se utilizara 100 resultados por pagina ya que es el limite que dispone la documentacion de la API y de esta manera , sera necesario acceder a las 14 paginas para completar el DF con todos los datos. Una manera de hacerlo es iterar sobre la utilizacion de la API y asi obtener la informacion completa.

In [53]:
import pandas as pd

#lista para almacenar los diccionarios de estadísticas
stats_list = []

for i in range (1,15):
    #global stats_list
    url = 'https://www.balldontlie.io/api/v1/games?seasons[]=2022&per_page=100&page=' + str(i)
    response = requests.get(url)
    data = response.json()
    # Iterar sobre cada partido en la pagina y agregar las estadísticas como columnas en el DataFrame
    for game in data['data']:
        stats = {'game_id': game['id'],
                'game_date' : game['date'],
                'game_home_team' : game['home_team']['name'],
                'game_home_team_score' : game['home_team_score'],
                'game_season' : game['season'],
                'game_visitor_team' : game['visitor_team']['name'],
                'game_visitor_team_score' : game['visitor_team_score']}
        stats_list.append(stats)
    
df = pd.DataFrame(stats_list)
df


,game_id,game_date,game_home_team,game_home_team_score,game_season,game_visitor_team,game_visitor_team_score
0,857369,2022-10-20T00:00:00.000Z,Lakers,97,2022,Clippers,103
1,857370,2022-10-20T00:00:00.000Z,76ers,88,2022,Bucks,90
2,857381,2022-10-21T00:00:00.000Z,Trail Blazers,113,2022,Suns,111
3,857374,2022-10-21T00:00:00.000Z,Wizards,102,2022,Bulls,100
4,857376,2022-10-21T00:00:00.000Z,Rockets,122,2022,Grizzlies,129
...,...,...,...,...,...,...,...
1302,981619,2023-05-08T00:00:00.000Z,Lakers,104,2022,Warriors,101
1303,981623,2023-05-10T00:00:00.000Z,Warriors,121,2022,Lakers,106
1304,981627,2023-05-12T00:00:00.000Z,Lakers,0,2022,Warriors,0
1305,1006710,2023-05-14T00:00:00.000Z,Warriors,0,2022,Lakers,0


Como se puede ver , el DF generado contiene los registros de toda la temporada 2022.

# Data Wrangling en datos actualizados

Se hara una limpieza y orden de los datos recientemente adquiridos para la futura utilizacion.

In [54]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1307 entries, 0 to 1306
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   game_id                  1307 non-null   int64 
 1   game_date                1307 non-null   object
 2   game_home_team           1307 non-null   object
 3   game_home_team_score     1307 non-null   int64 
 4   game_season              1307 non-null   int64 
 5   game_visitor_team        1307 non-null   object
 6   game_visitor_team_score  1307 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 71.6+ KB


Como condicion general , se busca que los datos numericos sean del tipo *float* o *int* .Tampoco se encontraron valores nulos entre los datos pero existe la posibilidad de que haya datos invalidos. Para eso se usara el metodo ".describe()" que realizara un analisis rapido estadisitico y asi saber si algun dato es incorrecto.

In [55]:
df.describe()

,game_id,game_home_team_score,game_season,game_visitor_team_score
count,1.307000e+03,1307.000000,1307.0,1307.000000
mean,8.618814e+05,115.085692,2022.0,112.449885
std,1.832810e+04,14.926374,0.0,14.851987
min,8.573550e+05,0.000000,2022.0,0.000000
25%,8.576815e+05,108.000000,2022.0,105.000000
50%,8.580080e+05,116.000000,2022.0,113.000000
75%,8.583345e+05,124.000000,2022.0,121.000000
max,1.006711e+06,175.000000,2022.0,176.000000


A pesar de que hay posibilidad de que existan partidos en los que haya 0 anotaciones (game_home_team_score , game_visitor_team_score) , es sumamente improbable. Por lo tanto se tomara las filas estas como **datos erroneos o faltantes**. Se contaran cuantos registros cumplen con estas condiciones

In [56]:
# Filtrar las filas que cumplen con la condición de que "game_home_team_score" sea igual a 0 o "game_visitor_team_score" sea igual a 0
filtered_df = df.loc[(df['game_home_team_score'] == 0) | (df['game_visitor_team_score'] == 0)]
filtered_df


,game_id,game_date,game_home_team,game_home_team_score,game_season,game_visitor_team,game_visitor_team_score
1288,970441,2023-05-12T00:00:00.000Z,Heat,0,2022,Knicks,0
1290,972866,2023-05-11T00:00:00.000Z,76ers,0,2022,Celtics,0
1296,967809,2023-05-11T00:00:00.000Z,Suns,0,2022,Nuggets,0
1297,972869,2023-05-14T00:00:00.000Z,Celtics,0,2022,76ers,0
1298,967810,2023-05-14T00:00:00.000Z,Nuggets,0,2022,Suns,0
1304,981627,2023-05-12T00:00:00.000Z,Lakers,0,2022,Warriors,0
1305,1006710,2023-05-14T00:00:00.000Z,Warriors,0,2022,Lakers,0
1306,1006711,2023-05-15T00:00:00.000Z,Knicks,0,2022,Heat,0


Como se puede ver , los casos que son nulos son pocos y no afectan realmente a los datos , por lo tanto , se procedera con la eliminacion de estos registros. El siguiente codigo es una manera distinta de eliminacion de los registros aunque igual de eficaz

In [57]:
df = df.loc[(df['game_home_team_score'] != 0) | (df['game_visitor_team_score'] != 0)]
df.describe()

,game_id,game_home_team_score,game_season,game_visitor_team_score
count,1299.000000,1299.000000,1299.0,1299.000000
mean,861148.689761,115.794457,2022.0,113.142417
std,15771.402805,11.917866,0.0,11.980231
min,857355.000000,80.000000,2022.0,79.000000
25%,857679.500000,108.000000,2022.0,105.000000
50%,858004.000000,116.000000,2022.0,113.000000
75%,858328.500000,124.000000,2022.0,121.000000
max,981623.000000,175.000000,2022.0,176.000000


Pasaremos a identificar casos duplicados, por ejemplo, partidos anotados dos veces. Una manera podria ser analizar la fecha de los partidos para ver si existen dos registros iguales en la columna *"game_date"*

In [58]:
#Realizamos la suma de las filas que estan duplicadas (True)
df_dup=sum(df.duplicated('game_date'))
df_dup

1106

A simple vista puede notarse que existen una gran cantidad de duplicados, demasiada representativa a la muestra total de los datos. Sera necesario distinguirlos con distinto criterio.

In [59]:
df

,game_id,game_date,game_home_team,game_home_team_score,game_season,game_visitor_team,game_visitor_team_score
0,857369,2022-10-20T00:00:00.000Z,Lakers,97,2022,Clippers,103
1,857370,2022-10-20T00:00:00.000Z,76ers,88,2022,Bucks,90
2,857381,2022-10-21T00:00:00.000Z,Trail Blazers,113,2022,Suns,111
3,857374,2022-10-21T00:00:00.000Z,Wizards,102,2022,Bulls,100
4,857376,2022-10-21T00:00:00.000Z,Rockets,122,2022,Grizzlies,129
...,...,...,...,...,...,...,...
1299,981609,2023-05-02T00:00:00.000Z,Warriors,112,2022,Lakers,117
1300,981611,2023-05-04T00:00:00.000Z,Warriors,127,2022,Lakers,100
1301,981615,2023-05-06T00:00:00.000Z,Lakers,127,2022,Warriors,97
1302,981619,2023-05-08T00:00:00.000Z,Lakers,104,2022,Warriors,101


In [60]:
#Realizamos la suma de las filas que estan duplicadas usando "game_id" (True)
df_dup=sum(df.duplicated('game_id'))
df_dup

0

Asi se puede verificar que el error se debe al problema en la anotacion de la fecha en "game_date" y no es correcto decir que existen duplicados ya que "game_id" demuestra que no hay repetidos

Este problema en la fecha puede traer confusion , por lo tanto , seria una buena conducta corregir el formato del dato a uno que provea informacion real. Es decir , se corregira el dato del horario ya que en todos los registros son **'T00:00:00.000Z'**.

Se usara el metodo **".apply"** en el df que almacena los datos para ejecutar una funcion que realice esta accion registro por registro.

In [61]:
# Función para eliminar el horario y dejar solo la fecha
def eliminar_horario(fecha):
    fecha_dt = pd.to_datetime(fecha)
    return fecha_dt.strftime('%Y-%m-%d')

# Aplicar la función a la columna 'fechas' utilizando apply
df['game_date'] = df['game_date'].apply(eliminar_horario)
df

C:\Users\Roberto\AppData\Local\Temp\ipykernel_14768\3196305801.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['game_date'] = df['game_date'].apply(eliminar_horario)


,game_id,game_date,game_home_team,game_home_team_score,game_season,game_visitor_team,game_visitor_team_score
0,857369,2022-10-20,Lakers,97,2022,Clippers,103
1,857370,2022-10-20,76ers,88,2022,Bucks,90
2,857381,2022-10-21,Trail Blazers,113,2022,Suns,111
3,857374,2022-10-21,Wizards,102,2022,Bulls,100
4,857376,2022-10-21,Rockets,122,2022,Grizzlies,129
...,...,...,...,...,...,...,...
1299,981609,2023-05-02,Warriors,112,2022,Lakers,117
1300,981611,2023-05-04,Warriors,127,2022,Lakers,100
1301,981615,2023-05-06,Lakers,127,2022,Warriors,97
1302,981619,2023-05-08,Lakers,104,2022,Warriors,101


De esta manera , los valores en "game_date" pasaran a ser el dia en el que se realizo el partido

Tambien nos interesa que sea un conjunto de datos ordenados (a pesar de estar relativamente ordenados , no siguen un orden completo) , asi , se buscara de ordenarlos por el "game_id" mostrando un orden especifico.

Para esta tarea se usara el metodo ".sort_values()" , que permitira ordenar las filas del dataframe de manera segun la caracteristica que consideremos apropiada ("game_id")

In [62]:
df = df.sort_values(by=['game_id'])
df

,game_id,game_date,game_home_team,game_home_team_score,game_season,game_visitor_team,game_visitor_team_score
1069,857355,2022-10-18,Celtics,126,2022,76ers,117
1073,857356,2022-10-18,Warriors,123,2022,Lakers,109
1016,857357,2022-10-19,Pacers,107,2022,Wizards,114
1038,857358,2022-10-19,Pistons,113,2022,Magic,109
953,857359,2022-10-19,Nets,108,2022,Pelicans,130
...,...,...,...,...,...,...,...
1299,981609,2023-05-02,Warriors,112,2022,Lakers,117
1300,981611,2023-05-04,Warriors,127,2022,Lakers,100
1301,981615,2023-05-06,Lakers,127,2022,Warriors,97
1302,981619,2023-05-08,Lakers,104,2022,Warriors,101


Como verificacion , se puede observar las diferencias que existen en el orden del indice del df. 

Como ultimo paso antes de exportarlo , se reordenara el indice para continuar con el correcto orden del DF

In [63]:
df = df.reset_index(drop=True)
df

,game_id,game_date,game_home_team,game_home_team_score,game_season,game_visitor_team,game_visitor_team_score
0,857355,2022-10-18,Celtics,126,2022,76ers,117
1,857356,2022-10-18,Warriors,123,2022,Lakers,109
2,857357,2022-10-19,Pacers,107,2022,Wizards,114
3,857358,2022-10-19,Pistons,113,2022,Magic,109
4,857359,2022-10-19,Nets,108,2022,Pelicans,130
...,...,...,...,...,...,...,...
1294,981609,2023-05-02,Warriors,112,2022,Lakers,117
1295,981611,2023-05-04,Warriors,127,2022,Lakers,100
1296,981615,2023-05-06,Lakers,127,2022,Warriors,97
1297,981619,2023-05-08,Lakers,104,2022,Warriors,101


Finalmente, el Dataframe ya posee datos utlizables y accesibles de manera confiable. 

Para poder utilizar estos datos sera necesario exportarlos en cualquiera de los formatos que sea conveniente. Continuando con los formatos anteriores, se exportara a un formato ".csv" para la posterior lectura del mismo.

In [64]:
# Exportar el DataFrame a un archivo CSV
df.to_csv('games_2022.csv', index=False)

Ahora que ya es un DataSet completo , debe ser necesario la descripcion del mismo:

## Introduccion de DataSet (games_2022.csv)

Este DataSet contiene los datos de los juegos de la NBA en la ultima temporada (2022). Este posee 1299 filas de datos y 7 columnas con los siguientes datos:

game_id: Identificación única del juego.

game_date: Fecha del juego en formato "YYYY-MM-DD".

game_home_team: Nombre del equipo local (equipo que juega en su propia cancha).

game_home_team_score: Puntuación del equipo local en ese juego.

game_season: Temporada del juego.

game_visitor_team: Nombre del equipo visitante (equipo que juega fuera de casa).

game_visitor_team_score: Puntuación del equipo visitante en ese juego.

Es posible que se desee guardar la descripcion rapida del .csv para un rapido acceso a la descripcion del mismo.